## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-16-10-44-05 +0000,bbc,Barclays fined £42m over money laundering checks,https://www.bbc.com/news/articles/cpwqeyj1d15o
1,2025-07-16-10-36-46 +0000,bbc,US and Indonesia reach tariff deal,https://www.bbc.com/news/articles/cwyq0ln6z6lo
2,2025-07-16-10-31-10 +0000,bbc,Trip drink ad banned over claim it makes you calm,https://www.bbc.com/news/articles/c14e2d2ednpo
3,2025-07-16-10-02-38 +0000,bbc,US says it has deported five 'barbaric' migran...,https://www.bbc.com/news/articles/clyze8mvzdgo
4,2025-07-16-09-58-10 +0000,bbc,Defence secretary 'unable to say' if anyone ki...,https://www.bbc.com/news/articles/c1k8yvj89kyo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2311/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
30,trump,8
40,inflation,5
139,president,5
234,canada,4
107,up,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
23,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...,28
22,2025-07-16-01-00-00 +0000,wsj,The impact of President Trump’s whirlwind six ...,https://www.wsj.com/economy/trump-effect-start...,26
31,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...,24
17,2025-07-16-06-35-00 +0000,wsj,U.K. Inflation Heats Up as Bank of England Sta...,https://www.wsj.com/economy/u-k-inflation-heat...,23
38,2025-07-15-16-09-00 +0000,wsj,"Canada Inflation Accelerates in June, Likely K...",https://www.wsj.com/economy/central-banking/ca...,20


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
23,28,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...
17,23,2025-07-16-06-35-00 +0000,wsj,U.K. Inflation Heats Up as Bank of England Sta...,https://www.wsj.com/economy/u-k-inflation-heat...
31,16,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...
33,15,2025-07-15-18-27-07 +0000,nyt,Canada Wildfire Smoke Triggers Air Quality Ale...,https://www.nytimes.com/2025/07/14/world/canad...
29,14,2025-07-15-20-10-21 +0000,bbc,Three key questions after Afghan data breach s...,https://www.bbc.com/news/articles/c9w12kdg5zko
12,13,2025-07-16-09-10-00 +0000,wsj,EU Exports to U.S. Cool as Trans-Atlantic Trad...,https://www.wsj.com/economy/trade/eu-exports-t...
40,12,2025-07-15-15-08-21 +0000,bbc,From military ruler to democrat - ex-Nigeria P...,https://www.bbc.com/news/articles/czxw2wn5v52o
28,11,2025-07-15-20-17-00 +0000,wsj,Leading Economies Must Work Together to Fix Im...,https://www.wsj.com/economy/global/leading-eco...
50,11,2025-07-15-11-27-08 +0000,bbc,"Thames Water 'extremely stressed', warns boss ...",https://www.bbc.com/news/articles/czxw2edxen9o
2,10,2025-07-16-10-31-10 +0000,bbc,Trip drink ad banned over claim it makes you calm,https://www.bbc.com/news/articles/c14e2d2ednpo


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
